# Chapter 16: Graph Traversals

> *"To traverse is to discover. In graphs, traversal algorithms reveal the structure, connectivity, and secrets hidden within the nodes and edges."* — Anonymous

---

## 16.1 Introduction to Graph Traversals

Graph traversal is the process of visiting all vertices in a graph in a systematic way. The two fundamental traversal strategies are:

- **Breadth-First Search (BFS)**: Explores the graph level by level, visiting all neighbors of a vertex before moving to the next level.
- **Depth-First Search (DFS)**: Explores as far as possible along each branch before backtracking.

Traversals form the backbone of countless graph algorithms, from connectivity analysis to shortest paths and beyond.

```
┌─────────────────────────────────────────────────────────────────────┐
│                    APPLICATIONS OF GRAPH TRAVERSALS                  │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  • CONNECTED COMPONENTS: Find all vertices reachable from a source  │
│  • PATH FINDING: Determine if a path exists between two vertices    │
│  • CYCLE DETECTION: Check if graph contains cycles                  │
│  • TOPOLOGICAL SORT: Order vertices in a DAG by dependencies        │
│  • STRONGLY CONNECTED COMPONENTS: Partition directed graph into     │
│    mutually reachable subgraphs                                      │
│  • BICONNECTED COMPONENTS: Find articulation points and bridges     │
│  • SHORTEST PATH (unweighted): BFS gives shortest path in edges     │
│  • PUZZLE SOLVING: BFS/DFS in state space search (e.g., sliding puzzles)│
│  • WEB CRAWLING: BFS to discover pages                               │
│  • SOCIAL NETWORK ANALYSIS: Degrees of separation (BFS)             │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 16.2 Breadth-First Search (BFS)

BFS explores a graph level by level, using a queue to maintain the order of visitation. It starts from a source vertex and visits all vertices at distance k before moving to distance k+1.

### 16.2.1 BFS Algorithm

```python
from collections import deque
from typing import List, Tuple, Optional

def bfs(graph, start):
    """
    Breadth-First Search traversal.
    
    Args:
        graph: adjacency list representation (list of lists of neighbors)
        start: starting vertex index
    
    Returns:
        visited: list of booleans indicating visited vertices
        distance: list of shortest distances from start (in edges)
        parent: list of parent vertices in BFS tree
    """
    n = len(graph)
    visited = [False] * n
    distance = [-1] * n
    parent = [-1] * n
    
    queue = deque([start])
    visited[start] = True
    distance[start] = 0
    
    while queue:
        u = queue.popleft()
        for v in graph[u]:
            if not visited[v]:
                visited[v] = True
                distance[v] = distance[u] + 1
                parent[v] = u
                queue.append(v)
    
    return visited, distance, parent


def bfs_traversal(graph, start):
    """Return list of vertices in BFS order starting from start."""
    visited, _, _ = bfs(graph, start)
    # BFS order is captured by the queue order; we can modify to record order
    # Here's a simple version that returns order:
    n = len(graph)
    visited = [False] * n
    order = []
    queue = deque([start])
    visited[start] = True
    while queue:
        u = queue.popleft()
        order.append(u)
        for v in graph[u]:
            if not visited[v]:
                visited[v] = True
                queue.append(v)
    return order
```

**Time Complexity:** O(V + E) – each vertex and edge processed once.  
**Space Complexity:** O(V) – queue and visited array.

### 16.2.2 Example

Consider this undirected graph:
```
0 -- 1 -- 2 -- 3
|    |         |
4 -- 5        6
```

Adjacency list:
```
0: [1, 4]
1: [0, 2, 5]
2: [1, 3]
3: [2, 6]
4: [0, 5]
5: [1, 4]
6: [3]
```

BFS from vertex 0:
```
Level 0: 0
Level 1: 1, 4
Level 2: 2, 5
Level 3: 3
Level 4: 6
```

```python
def demo_bfs():
    graph = [
        [1, 4],    # 0
        [0, 2, 5], # 1
        [1, 3],    # 2
        [2, 6],    # 3
        [0, 5],    # 4
        [1, 4],    # 5
        [3]        # 6
    ]
    visited, dist, parent = bfs(graph, 0)
    print("BFS order:", bfs_traversal(graph, 0))
    print("Distances from 0:", dist)
    print("Parents:", parent)

demo_bfs()
```

**Output:**
```
BFS order: [0, 1, 4, 2, 5, 3, 6]
Distances from 0: [0, 1, 2, 3, 1, 2, 4]
Parents: [-1, 0, 1, 2, 0, 1, 3]
```

### 16.2.3 Applications of BFS

#### 16.2.3.1 Shortest Path in Unweighted Graph

The `distance` array from BFS gives the shortest number of edges from the source to each vertex. To reconstruct the path to a target `t`:

```python
def shortest_path_bfs(graph, start, target):
    _, _, parent = bfs(graph, start)
    if parent[target] == -1 and start != target:
        return None  # no path
    path = []
    v = target
    while v != -1:
        path.append(v)
        v = parent[v]
    return path[::-1]
```

#### 16.2.3.2 Connected Components in Undirected Graph

BFS (or DFS) can label all connected components:

```python
def connected_components_bfs(graph):
    n = len(graph)
    visited = [False] * n
    components = []
    for v in range(n):
        if not visited[v]:
            comp = []
            queue = deque([v])
            visited[v] = True
            while queue:
                u = queue.popleft()
                comp.append(u)
                for w in graph[u]:
                    if not visited[w]:
                        visited[w] = True
                        queue.append(w)
            components.append(comp)
    return components
```

#### 16.2.3.3 Bipartite Check

A graph is bipartite if and only if it can be 2-colored. BFS can assign colors level by level:

```python
def is_bipartite_bfs(graph):
    n = len(graph)
    color = [-1] * n  # -1 uncolored, 0 and 1 colors
    for start in range(n):
        if color[start] == -1:
            queue = deque([start])
            color[start] = 0
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if color[v] == -1:
                        color[v] = 1 - color[u]
                        queue.append(v)
                    elif color[v] == color[u]:
                        return False
    return True
```

#### 16.2.3.4 Word Ladder Problem

Given two words of equal length and a dictionary, find the shortest transformation sequence where each step changes one letter and intermediate words are in the dictionary.

**Solution:** Model as graph where words are vertices, edges connect words differing by one letter. BFS finds shortest path.

---

## 16.3 Depth-First Search (DFS)

DFS explores a graph by going as deep as possible from a vertex, backtracking only when no further unvisited neighbors exist.

### 16.3.1 Recursive DFS

```python
def dfs_recursive(graph, start, visited=None, parent=None, order=None):
    if visited is None:
        visited = [False] * len(graph)
        parent = [-1] * len(graph)
        order = []
    visited[start] = True
    order.append(start)
    for v in graph[start]:
        if not visited[v]:
            parent[v] = start
            dfs_recursive(graph, v, visited, parent, order)
    return visited, parent, order
```

### 16.3.2 Iterative DFS (using stack)

```python
def dfs_iterative(graph, start):
    n = len(graph)
    visited = [False] * n
    parent = [-1] * n
    stack = [start]
    order = []
    while stack:
        u = stack.pop()
        if not visited[u]:
            visited[u] = True
            order.append(u)
            # Push neighbors in reverse order to simulate recursive order
            for v in reversed(graph[u]):
                if not visited[v]:
                    parent[v] = u
                    stack.append(v)
    return visited, parent, order
```

**Time Complexity:** O(V + E) – each vertex and edge processed once.  
**Space Complexity:** O(V) – stack and visited array.

### 16.3.3 Example

Using the same graph as before, DFS from vertex 0 (recursive order depends on neighbor order; assume adjacency lists as given):

```
Order: 0, 1, 2, 3, 6, 5, 4
```

```python
def demo_dfs():
    graph = [
        [1, 4],    # 0
        [0, 2, 5], # 1
        [1, 3],    # 2
        [2, 6],    # 3
        [0, 5],    # 4
        [1, 4],    # 5
        [3]        # 6
    ]
    _, _, order = dfs_recursive(graph, 0)
    print("DFS recursive order:", order)
    _, _, order_it = dfs_iterative(graph, 0)
    print("DFS iterative order:", order_it)

demo_dfs()
```

**Output:**
```
DFS recursive order: [0, 1, 2, 3, 6, 5, 4]
DFS iterative order: [0, 4, 5, 1, 2, 3, 6]   # order may differ
```

### 16.3.4 Applications of DFS

#### 16.3.4.1 Cycle Detection

- **Undirected graph:** If during DFS we encounter an already visited vertex that is not the parent, there is a cycle.

```python
def has_cycle_undirected_dfs(graph):
    n = len(graph)
    visited = [False] * n
    def dfs(u, parent):
        visited[u] = True
        for v in graph[u]:
            if not visited[v]:
                if dfs(v, u):
                    return True
            elif v != parent:
                return True
        return False
    for i in range(n):
        if not visited[i]:
            if dfs(i, -1):
                return True
    return False
```

- **Directed graph:** Use recursion stack (or three-color method) to detect back edges.

```python
def has_cycle_directed_dfs(graph):
    n = len(graph)
    state = [0] * n  # 0=unvisited, 1=visiting, 2=processed
    def dfs(u):
        state[u] = 1
        for v in graph[u]:
            if state[v] == 1:  # back edge
                return True
            if state[v] == 0 and dfs(v):
                return True
        state[u] = 2
        return False
    for i in range(n):
        if state[i] == 0:
            if dfs(i):
                return True
    return False
```

#### 16.3.4.2 Topological Sorting (Preview)

DFS can produce a topological order of a DAG by appending vertices to a list after processing all descendants (postorder), then reversing.

#### 16.3.4.3 Connected Components

Similar to BFS, DFS can label connected components.

#### 16.3.4.4 Path Finding

DFS can find a path between two vertices, but not necessarily the shortest.

#### 16.3.4.5 Solving Puzzles (Maze, N-Queens)

DFS is natural for exploring state spaces.

---

## 16.4 Topological Sorting

A **topological sort** of a directed acyclic graph (DAG) is a linear ordering of vertices such that for every directed edge u → v, u comes before v in the ordering. Topological sorts are used in scheduling tasks with dependencies.

### 16.4.1 Kahn's Algorithm (BFS-based)

Kahn's algorithm uses indegrees and a queue.

```python
from collections import deque

def topological_sort_kahn(graph):
    """
    graph: adjacency list (directed)
    Returns list of vertices in topological order, or None if cycle exists.
    """
    n = len(graph)
    indegree = [0] * n
    for u in range(n):
        for v in graph[u]:
            indegree[v] += 1

    queue = deque([u for u in range(n) if indegree[u] == 0])
    top_order = []

    while queue:
        u = queue.popleft()
        top_order.append(u)
        for v in graph[u]:
            indegree[v] -= 1
            if indegree[v] == 0:
                queue.append(v)

    if len(top_order) != n:
        return None  # cycle detected
    return top_order
```

**Time Complexity:** O(V + E)

### 16.4.2 DFS-based Topological Sort

Perform DFS, and after finishing a vertex (postorder), add it to a stack. The stack, when popped, gives topological order.

```python
def topological_sort_dfs(graph):
    n = len(graph)
    visited = [False] * n
    stack = []

    def dfs(u):
        visited[u] = True
        for v in graph[u]:
            if not visited[v]:
                dfs(v)
        stack.append(u)  # postorder

    for i in range(n):
        if not visited[i]:
            dfs(i)

    return stack[::-1]  # reverse to get correct order
```

**Cycle detection** can be incorporated using three-state coloring.

### 16.4.3 Example

Consider a DAG representing course prerequisites:
```
0 → 1 → 3
↓   ↓
2 → 4
```
Adjacency list:
```
0: [1,2]
1: [3,4]
2: [4]
3: []
4: []
```

Topological order: [0, 2, 1, 4, 3] or [0, 1, 2, 4, 3], etc.

```python
def demo_topological():
    graph = [
        [1, 2],
        [3, 4],
        [4],
        [],
        []
    ]
    print("Kahn:", topological_sort_kahn(graph))
    print("DFS:", topological_sort_dfs(graph))

demo_topological()
```

**Output:**
```
Kahn: [0, 1, 2, 3, 4]   # depends on queue order
DFS: [0, 2, 1, 4, 3]
```

### 16.4.4 Applications

- **Build systems** (Make, Bazel): Compile dependencies first.
- **Course scheduling**: Determine order of courses respecting prerequisites.
- **Data processing pipelines**: Execute stages in correct order.
- **Spreadsheet formula evaluation**: Evaluate cells with dependencies.

---

## 16.5 Strongly Connected Components

In a directed graph, a **strongly connected component (SCC)** is a maximal set of vertices where every vertex is reachable from every other within the set. The graph of SCCs (condensation) is a DAG.

### 16.5.1 Kosaraju's Algorithm

Kosaraju's algorithm uses two DFS passes:

1. Perform DFS on the original graph to record finishing times (postorder).
2. Reverse the graph (transpose).
3. Process vertices in decreasing order of finishing times, running DFS on the reversed graph; each DFS tree gives one SCC.

```python
def kosaraju_scc(graph):
    n = len(graph)
    visited = [False] * n
    order = []

    # First DFS to record finishing order
    def dfs1(u):
        visited[u] = True
        for v in graph[u]:
            if not visited[v]:
                dfs1(v)
        order.append(u)

    for i in range(n):
        if not visited[i]:
            dfs1(i)

    # Build reversed graph
    rev_graph = [[] for _ in range(n)]
    for u in range(n):
        for v in graph[u]:
            rev_graph[v].append(u)

    # Second DFS on reversed graph
    visited2 = [False] * n
    sccs = []

    def dfs2(u, component):
        visited2[u] = True
        component.append(u)
        for v in rev_graph[u]:
            if not visited2[v]:
                dfs2(v, component)

    for u in reversed(order):  # process in decreasing finish time
        if not visited2[u]:
            component = []
            dfs2(u, component)
            sccs.append(component)

    return sccs
```

**Time Complexity:** O(V + E)

### 16.5.2 Tarjan's Algorithm

Tarjan's algorithm finds SCCs in a single DFS, using a stack and low-link values. It's more efficient in practice but slightly more complex.

```python
def tarjan_scc(graph):
    n = len(graph)
    index = 0
    indices = [-1] * n
    lowlink = [-1] * n
    on_stack = [False] * n
    stack = []
    result = []

    def strongconnect(v):
        nonlocal index
        indices[v] = index
        lowlink[v] = index
        index += 1
        stack.append(v)
        on_stack[v] = True

        for w in graph[v]:
            if indices[w] == -1:
                strongconnect(w)
                lowlink[v] = min(lowlink[v], lowlink[w])
            elif on_stack[w]:
                lowlink[v] = min(lowlink[v], indices[w])

        if lowlink[v] == indices[v]:  # root of SCC
            scc = []
            while True:
                w = stack.pop()
                on_stack[w] = False
                scc.append(w)
                if w == v:
                    break
            result.append(scc)

    for v in range(n):
        if indices[v] == -1:
            strongconnect(v)
    return result
```

### 16.5.3 Example

Graph:
```
0 → 1 → 2 → 0
    ↓   ↓
    3 → 4
```
SCCs: {0,1,2}, {3}, {4}

```python
def demo_scc():
    graph = [
        [1],
        [2, 3],
        [0],
        [4],
        []
    ]
    print("Kosaraju SCCs:", kosaraju_scc(graph))
    print("Tarjan SCCs:", tarjan_scc(graph))

demo_scc()
```

### 16.5.4 Applications

- **Graph condensation**: Collapse SCCs to form a DAG, enabling topological algorithms.
- **2-SAT problem**: Solve by constructing implication graph and finding SCCs.
- **Social networks**: Find communities with mutual following.
- **Program analysis**: Detect cycles in call graphs.

---

## 16.6 Biconnected Components and Articulation Points

In an undirected graph, an **articulation point** (or cut vertex) is a vertex whose removal increases the number of connected components. A **bridge** (cut edge) is an edge whose removal increases components. **Biconnected components** are maximal sets of edges where any two edges lie on a common simple cycle (i.e., removal of any single vertex does not disconnect the component).

### 16.6.1 Finding Articulation Points and Bridges (Tarjan's Algorithm)

Use DFS with discovery times and low-link values.

```python
def find_articulation_points_and_bridges(graph):
    n = len(graph)
    ids = [-1] * n
    low = [0] * n
    visited = [False] * n
    articulation = [False] * n
    bridges = []
    id_counter = 0

    def dfs(at, parent, parent_edge):
        nonlocal id_counter
        visited[at] = True
        ids[at] = low[at] = id_counter
        id_counter += 1

        children = 0
        for to in graph[at]:
            if to == parent:
                continue
            if not visited[to]:
                children += 1
                dfs(to, at, (at, to))
                low[at] = min(low[at], low[to])

                # Articulation point conditions
                if parent != -1 and low[to] >= ids[at]:
                    articulation[at] = True
                # Bridge condition
                if low[to] > ids[at]:
                    bridges.append((at, to))
            else:
                low[at] = min(low[at], ids[to])

        # Root articulation point
        if parent == -1 and children > 1:
            articulation[at] = True

    for i in range(n):
        if not visited[i]:
            dfs(i, -1, None)

    return articulation, bridges
```

**Time Complexity:** O(V + E)

### 16.6.2 Example

Graph:
```
0 -- 1 -- 2
|    |
3 -- 4
```
Articulation points: 1 (if 0,3,4 connected only through 1). Bridges: (1,2) if 2 is leaf.

### 16.6.3 Applications

- **Network reliability**: Identify critical routers or links.
- **Graph visualization**: Biconnected components reveal cycles.
- **Ear decomposition**: Used in graph theory.

---

## 16.7 Summary and Comparisons

```
┌─────────────────────────────────────────────────────────────────────┐
│                    GRAPH TRAVERSAL ALGORITHMS                        │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Algorithm              │ Time      │ Space     │ Use Case           │
│─────────────────────────┼───────────┼───────────┼────────────────────┤
│ BFS                     │ O(V+E)    │ O(V)      │ Shortest path (unweighted),│
│                         │           │           │ level-order         │
│ DFS                     │ O(V+E)    │ O(V)      │ Connectivity, cycles,│
│                         │           │           │ topological (DAG)   │
│ Topological (Kahn)      │ O(V+E)    │ O(V)      │ Task scheduling     │
│ Topological (DFS)       │ O(V+E)    │ O(V)      │ DAG ordering        │
│ Kosaraju SCC            │ O(V+E)    │ O(V)      │ Strong components   │
│ Tarjan SCC              │ O(V+E)    │ O(V)      │ Strong components   │
│ Articulation/Bridges    │ O(V+E)    │ O(V)      │ Network reliability │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 16.8 Practice Problems

### Problem 1: Number of Islands (LeetCode 200)
Given a 2D grid of '1's (land) and '0's (water), count the number of islands (connected components of '1's).

**Hint:** BFS/DFS on grid cells.

### Problem 2: Course Schedule (LeetCode 207)
There are a total of n courses labeled from 0 to n-1. You are given prerequisites pairs [a, b] meaning you must take course b before course a. Determine if you can finish all courses.

**Hint:** Detect cycle in directed graph; use topological sort.

### Problem 3: Word Ladder (LeetCode 127)
Given two words (beginWord and endWord) and a dictionary, find the length of shortest transformation sequence.

**Hint:** BFS on graph where edges connect words differing by one letter.

### Problem 4: Critical Connections in a Network (LeetCode 1192)
Find all bridges in a graph.

**Hint:** Tarjan's bridge-finding algorithm.

### Problem 5: Evaluate Division (LeetCode 399)
Given equations like "a / b = 2.0" and queries like "a / c = ?", answer queries or return -1.0 if unknown.

**Hint:** Build graph with variables as nodes and division as weighted edges; use DFS/BFS to find path.

### Problem 6: Alien Dictionary (LeetCode 269)
Given a sorted dictionary of an alien language, find the order of characters.

**Hint:** Build graph from adjacent words; topological sort.

### Problem 7: Minimum Height Trees (LeetCode 310)
Find roots that minimize height in undirected tree.

**Hint:** Repeatedly remove leaves (BFS from leaves) until 1 or 2 nodes remain.

### Problem 8: Pacific Atlantic Water Flow (LeetCode 417)
Given a matrix of heights, find cells from which water can flow to both Pacific and Atlantic oceans.

**Hint:** Reverse BFS/DFS from oceans.

---

## 16.9 Further Reading

1. **"Introduction to Algorithms" (CLRS)** – Chapter 22 (Elementary Graph Algorithms)
2. **"Algorithms"** by Robert Sedgewick – Chapter 4 (Graphs)
3. **"The Algorithm Design Manual"** by Steven Skiena – Chapter 5 (Graph Traversal)
4. **"Graph Theory with Applications"** by Bondy and Murty
5. **Original Papers**:
   - Tarjan, R. (1972) – "Depth-First Search and Linear Graph Algorithms"
   - Kosaraju, S. R. (1978) – Unpublished (SCC algorithm)
   - Hopcroft, J., & Tarjan, R. (1973) – "Algorithm 447: Efficient algorithms for graph manipulation"

---

> **Coming in Chapter 17**: **Shortest Path Algorithms** – We'll explore Dijkstra, Bellman-Ford, Floyd-Warshall, and their variants.

---

**End of Chapter 16**